# 📖 Notebook 1 - Ingestion et Nettoyage des Textes Harry Potter

Ce notebook extrait et nettoie les textes des livres Harry Potter depuis les PDFs.

## Objectifs
1. Extraire le texte brut des 7 livres PDF
2. Nettoyer et normaliser le texte
3. Segmenter par phrases et dialogues
4. Exporter en format parquet pour analyse ultérieure

## Sorties
- `data/sentences.parquet` : corpus segmenté par phrases
- `data/book_metadata.json` : métadonnées des livres

In [1]:
# Imports
import os
import re
import json
from pathlib import Path
from typing import Dict, List, Tuple
import PyPDF2
import pandas as pd
import numpy as np
from tqdm import tqdm

# Configuration des chemins
NOTEBOOK_DIR = Path.cwd()

# Trouver la racine du dépôt en cherchant le dossier 'context'
def find_repo_root(start: Path) -> Path:
    for p in [start] + list(start.parents):
        if (p / 'context').is_dir():
            return p
    return start

REPO_ROOT = find_repo_root(NOTEBOOK_DIR)
PROJECT_DIR = None
# Trouver le répertoire de projet (contenant 'notebooks')
for p in [NOTEBOOK_DIR] + list(NOTEBOOK_DIR.parents):
    if (p / 'notebooks').is_dir():
        PROJECT_DIR = p
        break
if PROJECT_DIR is None:
    PROJECT_DIR = NOTEBOOK_DIR

BOOKS_DIR = REPO_ROOT / 'context' / 'books'
DATA_DIR = PROJECT_DIR / 'data'

# Créer le dossier data si nécessaire
DATA_DIR.mkdir(parents=True, exist_ok=True)

print(f"📁 Repo root: {REPO_ROOT}")
print(f"📁 Project dir: {PROJECT_DIR}")
print(f"📚 Books directory: {BOOKS_DIR}")
print(f"💾 Data output: {DATA_DIR}")

📁 Repo root: C:\Users\julie\src\School\Workshop\workshop-poudlard-epsi
📁 Project dir: C:\Users\julie\src\School\Workshop\workshop-poudlard-epsi\projects\22-proces-jk-rowling\hp_nlp
📚 Books directory: C:\Users\julie\src\School\Workshop\workshop-poudlard-epsi\context\books
💾 Data output: C:\Users\julie\src\School\Workshop\workshop-poudlard-epsi\projects\22-proces-jk-rowling\hp_nlp\data


In [2]:
# Métadonnées des livres
BOOK_INFO = {
    "harry-potter-1-lecole-des-sorciers.pdf": {
        "title": "L'École des Sorciers",
        "book_number": 1,
        "pages": 320,
        "year": 1997
    },
    "harry-potter-2-la-chambre-des-secrets.pdf": {
        "title": "La Chambre des Secrets",
        "book_number": 2,
        "pages": 360,
        "year": 1998
    },
    "harry-potter-3-le-prisonnier-dazkaban.pdf": {
        "title": "Le Prisonnier d'Azkaban",
        "book_number": 3,
        "pages": 420,
        "year": 1999
    },
    "harry-potter-4-la-coupe-de-feu.pdf": {
        "title": "La Coupe de Feu",
        "book_number": 4,
        "pages": 656,
        "year": 2000
    },
    "harry-potter-5-lordre-du-phoenix.pdf": {
        "title": "L'Ordre du Phénix",
        "book_number": 5,
        "pages": 980,
        "year": 2003
    },
    "harry-potter-6-le-prince-de-sang-mecc82lecc81.pdf": {
        "title": "Le Prince de Sang-Mêlé",
        "book_number": 6,
        "pages": 640,
        "year": 2005
    },
    "harry-potter-7-les-reliques-de-la-mort.pdf": {
        "title": "Les Reliques de la Mort",
        "book_number": 7,
        "pages": 800,
        "year": 2007
    }
}

print(f"📚 Nombre de livres configurés: {len(BOOK_INFO)}")

📚 Nombre de livres configurés: 7


## 1. Extraction du texte depuis les PDFs

In [3]:
def extract_text_from_pdf(pdf_path: Path) -> str:
    """Extrait le texte d'un fichier PDF."""
    text = ""
    
    try:
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            print(f"  📄 Pages: {len(pdf_reader.pages)}")
            
            for page_num, page in enumerate(tqdm(pdf_reader.pages, desc="Extraction")):
                try:
                    text += page.extract_text() + "\n"
                except Exception as e:
                    print(f"  ⚠️  Erreur page {page_num}: {e}")
                    continue
    except Exception as e:
        print(f"  ❌ Erreur lecture PDF: {e}")
        return ""
    
    return text


def clean_text(text: str) -> str:
    """Nettoie le texte extrait."""
    # Supprimer les sauts de ligne excessifs
    text = re.sub(r'\n{3,}', '\n\n', text)
    
    # Supprimer les espaces multiples
    text = re.sub(r' +', ' ', text)
    
    # Normaliser les guillemets
    text = text.replace('\u2018', "'").replace('\u2019', "'")
    text = text.replace('\u201c', '«').replace('\u201d', '»')
    
    return text.strip()

In [4]:
# Extraire le texte de tous les livres
books_data = []

for filename, info in BOOK_INFO.items():
    pdf_path = BOOKS_DIR / filename
    
    if not pdf_path.exists():
        print(f"⚠️  Livre non trouvé: {filename}")
        continue
    
    print(f"\n📖 Extraction: {info['title']}")
    
    # Extraire et nettoyer
    raw_text = extract_text_from_pdf(pdf_path)
    clean = clean_text(raw_text)
    
    # Statistiques de base
    word_count = len(clean.split())
    char_count = len(clean)
    
    print(f"  ✅ Mots: {word_count:,}")
    print(f"  ✅ Caractères: {char_count:,}")
    
    books_data.append({
        'filename': filename,
        'book_number': info['book_number'],
        'title': info['title'],
        'pages': info['pages'],
        'year': info['year'],
        'text': clean,
        'word_count': word_count,
        'char_count': char_count
    })

print(f"\n✅ Total livres extraits: {len(books_data)}")


📖 Extraction: L'École des Sorciers
  📄 Pages: 208


Extraction:   0%|                                                                                                                  | 0/208 [00:00<?, ?it/s]

Extraction:   2%|██                                                                                                        | 4/208 [00:00<00:06, 32.47it/s]

Extraction:   4%|████                                                                                                      | 8/208 [00:00<00:06, 29.28it/s]

Extraction:   6%|██████                                                                                                   | 12/208 [00:00<00:06, 32.07it/s]

...

Extraction:  98%|██████████████████████████████████████████████████████████████████████████████████████████████████████  | 531/541 [00:17<00:00, 28.19it/s]

Extraction:  99%|██████████████████████████████████████████████████████████████████████████████████████████████████████▋ | 534/541 [00:17<00:00, 27.98it/s]

Extraction:  99%|███████████████████████████████████████████████████████████████████████████████████████████████████████▏| 537/541 [00:17<00:00, 28.03it/s]

Extraction: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 541/541 [00:17<00:00, 30.32it/s]

  ✅ Mots: 230,278
  ✅ Caractères: 1,340,353

✅ Total livres extraits: 7


## 2. Segmentation en phrases

In [5]:
def segment_into_sentences(text: str) -> List[str]:
    """Segmente le texte en phrases.
    
    Note: Pour une segmentation plus précise, on utilisera spaCy dans le notebook 02.
    Ici, on fait une segmentation simple basée sur les points.
    """
    # Regex simple pour diviser sur . ! ? suivis d'espace et majuscule
    sentences = re.split(r'(?<=[.!?])\s+(?=[A-ZÀ-Ü])', text)
    
    # Filtrer les phrases trop courtes
    sentences = [s.strip() for s in sentences if len(s.strip()) > 10]
    
    return sentences

In [6]:
# Créer le corpus de phrases
all_sentences = []

for book in books_data:
    print(f"\n📝 Segmentation: {book['title']}")
    
    sentences = segment_into_sentences(book['text'])
    
    for idx, sentence in enumerate(sentences):
        all_sentences.append({
            'book_number': book['book_number'],
            'book_title': book['title'],
            'sentence_id': idx,
            'text': sentence,
            'length': len(sentence)
        })
    
    print(f"  ✅ {len(sentences):,} phrases extraites")

print(f"\n✅ Total phrases: {len(all_sentences):,}")


📝 Segmentation: L'École des Sorciers
  ✅ 4,934 phrases extraites

📝 Segmentation: La Chambre des Secrets
  ✅ 4,951 phrases extraites

📝 Segmentation: Le Prisonnier d'Azkaban
  ✅ 6,742 phrases extraites

📝 Segmentation: La Coupe de Feu
  ✅ 11,134 phrases extraites

📝 Segmentation: L'Ordre du Phénix
  ✅ 12,008 phrases extraites

📝 Segmentation: Le Prince de Sang-Mêlé
  ✅ 8,361 phrases extraites

📝 Segmentation: Les Reliques de la Mort
  ✅ 10,524 phrases extraites

✅ Total phrases: 58,654


## 3. Export des données

In [7]:
# Créer DataFrame et préparer l'export en parquet
df_sentences = pd.DataFrame(all_sentences)

# Définir le chemin de sortie même si on n'exporte pas (utilisé plus tard)
output_path = DATA_DIR / 'sentences.parquet'

try:
    if df_sentences.empty:
        print("\n⚠️ Aucune phrase extraite — vérifiez que les PDFs existent dans the 'context/books' directory.")
        print(f"  Books dir checked: {BOOKS_DIR}")
        print("✅ Aucune exportation réalisée.")
    else:
        print("\n📊 Aperçu du DataFrame:")
        print(df_sentences.head())
        print(f"\n📏 Shape: {df_sentences.shape}")
        print(f"\n📈 Statistiques:")
        try:
            print(df_sentences.groupby('book_title')['sentence_id'].count())
        except Exception as e:
            print(f"⚠️ Impossible d'obtenir les statistiques de groupby: {e}")
except Exception as e:
    print(f"⚠️ Erreur lors de la préparation du DataFrame: {e}")


📊 Aperçu du DataFrame:


   book_number            book_title  sentence_id  \
0            1  L'École des Sorciers            0   
1            1  L'École des Sorciers            1   
2            1  L'École des Sorciers            2   
3            1  L'École des Sorciers            3   
4            1  L'École des Sorciers            4   

                                                text  length  
0                                    L'auteure\nJ.K.      14  
1  Rowling\test\tnée\ten\t1967\tet\ta\tpassé\tson...     101  
2  Elle\ta\tsuivi\tdes\tétudes\tà\tl'université\t...     100  
3  Elle\ta\tensuite\ttravaillé\tquelque\ttemps\tà...     122  
4  C'est\ten\t1990\tque\tl'idée\tde\tHarry\tPotte...     160  

📏 Shape: (58654, 5)

📈 Statistiques:
book_title
L'Ordre du Phénix          12008
L'École des Sorciers        4934
La Chambre des Secrets      4951
La Coupe de Feu            11134
Le Prince de Sang-Mêlé      8361
Le Prisonnier d'Azkaban     6742
Les Reliques de la Mort    10524
Name: sentence_id, dtyp

In [8]:
# Exporter en parquet si des phrases existent
if 'df_sentences' in globals() and not df_sentences.empty:
    output_path = DATA_DIR / 'sentences.parquet'
    df_sentences.to_parquet(output_path, index=False)
    print(f"✅ Données exportées: {output_path}")
    try:
        print(f"📦 Taille fichier: {output_path.stat().st_size / 1024 / 1024:.2f} MB")
    except Exception:
        pass
else:
    print("⚠️ Pas d'export car aucun texte n'a été extrait.")

✅ Données exportées: C:\Users\julie\src\School\Workshop\workshop-poudlard-epsi\projects\22-proces-jk-rowling\hp_nlp\data\sentences.parquet
📦 Taille fichier: 4.74 MB


In [9]:
# Exporter les métadonnées
metadata = {
    'books': [
        {
            'book_number': book['book_number'],
            'title': book['title'],
            'pages': book['pages'],
            'year': book['year'],
            'word_count': book['word_count'],
            'char_count': book['char_count']
        }
        for book in books_data
    ],
    'total_sentences': len(all_sentences),
    'extraction_date': pd.Timestamp.now().isoformat()
}

metadata_path = DATA_DIR / 'book_metadata.json'
with open(metadata_path, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)

print(f"✅ Métadonnées exportées: {metadata_path}")

✅ Métadonnées exportées: C:\Users\julie\src\School\Workshop\workshop-poudlard-epsi\projects\22-proces-jk-rowling\hp_nlp\data\book_metadata.json


## 4. Validation des données

In [10]:
# Vérifier que les données peuvent être relues (si le fichier existe)
if output_path.exists():
    df_test = pd.read_parquet(output_path)
    print(f"✅ Vérification lecture parquet: {df_test.shape}")

    # Afficher quelques exemples
    print("\n📖 Exemples de phrases:")
    for i in range(min(5, len(df_test))):
        row = df_test.iloc[i]
        print(f"\nLivre {row['book_number']} - {row['book_title']}")
        print(f"  {row['text'][:200]}...")
else:
    print("⚠️ Aucun fichier parquet trouvé — pas d'extraction valide.")

✅ Vérification lecture parquet: (58654, 5)

📖 Exemples de phrases:

Livre 1 - L'École des Sorciers
  L'auteure
J.K....

Livre 1 - L'École des Sorciers
  Rowling	est	née	en	1967	et	a	passé	son	enfance	à	Chepstow,	dans	le	comté	de	Gwent,	au	pays	de
Galles....

Livre 1 - L'École des Sorciers
  Elle	a	suivi	des	études	à	l'université	d'Exeter	et	est	diplômée	en	langue	et	littérature	françaises....

Livre 1 - L'École des Sorciers
  Elle	a	ensuite	travaillé	quelque	temps	à	Londres	au	sein	de	l'association	Amnesty	International	et	a
enseigné	le	français....

Livre 1 - L'École des Sorciers
  C'est	en	1990	que	l'idée	de	Harry	Potter	et	de	son	école	de	magiciens	a	commencé	à	germer	dans	son
esprit,	alors	qu'elle	attendait	un	train	qui	avait	du	retard....


## ✅ Résumé

Ce notebook a:
1. ✅ Extrait le texte des 7 livres Harry Potter
2. ✅ Nettoyé et normalisé le texte
3. ✅ Segmenté en phrases
4. ✅ Exporté en format parquet

**Prochaine étape**: Notebook 02 - Pipeline NLP pour NER, coréférence et attribution de locuteur